## Publish LoRaWAN payloads to a myDevices Cayenne web dashboard

### Introduction

In the [LoRaWAN getting-started Notebook](../getting-started/lorawan-getting-started.ipynb), we learnt how to use LoRa boards available on IoT-LAB with [TheThingsNetwork LoRaWAN provider](https://www.thethingsnetwork.org/) (TTN). In addition to send/receive messages to/from the TTN backend, it is also possible to configure [integrations](https://www.thethingsnetwork.org/docs/applications/integrations.html) to a LoRaWAN network. One of these integrations is [myDevices Cayenne](https://www.thethingsnetwork.org/docs/applications/cayenne/), which allows to retrieve and display sensors data in customizable dashboards.

- The first step of this Notebook is to configure the integration between a TTN application and myDevices Cayenne. This is explained in the next section.
- The second step will be to implement the RIOT application. This is explained in the "Write your RIOT application" section.
- The last step will guide you in the submission of an IoT-LAB experiment with one LoRa device

### Configure the integration between a TTN application and myDevices Cayenne

If you followed the **[LoRaWAN getting-started Notebook](../getting-started/lorawan-getting-started.ipynb)**, the only thing that you have to do is to configure the integration of your TTN application with myDevices Cayenne: this is simple, just follow **[this documentation](https://www.thethingsnetwork.org/docs/applications/cayenne/)**.

### Write your RIOT application

myDevices Cayenne requires the devices to encode the messages in a specific payload format called **[Cayenne LPP](https://developers.mydevices.com/cayenne/docs/lora/#lora-cayenne-low-power-payload)**. This format is a binary format designed to reduce the payload size (remember that LoRaWAN has a very low datarate) while remaining quite flexible: the format encodes several channels that contain different kind of sendor data types (temperature, relative humidity, etc).

In the same directory as this notebook, you will find the 2 files of a basic RIOT application: `Makefile`, `main.c`. This application already contains code to read sensors, join a LoRaWAN and send a basic payload every 20s.

You can double click the files in the JupyterLab browser to open them in new tabs and start editing them.

As you can see, the application is already quite complete but the DEVEUI/APPEUI/APPKEY are empty (full of zeros) and the encoding to the Cayenne LPP format is missing in `main.c`. Fortunately, RIOT provides support for [a library for encoding to the Cayenne LPP](https://github.com/aabadie/cayenne-lpp). This is what will be used in this Notebook.

1. Change the DEVEUI, APPEUI, APPKEY default values (see C static arrays lines 28 to 30 in main.c). These values are available in your TTN device management page.

  **Note**: in the TTN device management page, you can switch the DEVEUI/APPEUI/APPKEY values from a plain hexadecimal string representation to a C array representation by clicking the `<>` button. Keep the _msb_ representation.
  
  **Important**: if you don't edit the deveui/appeui/appkey values, you device won't be able to join the network and send messages.

2. Add the missing Cayenne LPP header (under `/* TODO: Add the cayenne_lpp header */`):

```c
#include "cayenne_lpp.h"
```

3. Declare globally Cayenne LPP descriptor:

```c
static cayenne_lpp_t lpp;
```

4. In the sender function, prepare the cayenne lpp payload with the temperature and relative humidity values measured:

```c
cayenne_lpp_add_temperature(&lpp, 0, (float)temperature / 10);
cayenne_lpp_add_relative_humidity(&lpp, 1, (float)humidity / 10);
```

5. You have to replace the send of the dummy payload with the send of the cayenne lpp payload, e.g:

```
uint8_t ret = semtech_loramac_send(&loramac, lpp.buffer, lpp.cursor);
```

instead of

```
uint8_t ret = semtech_loramac_send(&loramac, (uint8_t *)message, strlen(message));
```

6. At the end of the while loop, you have to clear the cayenne lpp payload buffer between each send:

```c
cayenne_lpp_reset(&lpp);
```

7. You can now build the application:


In [ ]:
!make

### Experiment on IoT-LAB

1. Submit an experiment on IoT-LAB

In [ ]:
!iotlab-experiment submit -n "riot-cayenne-lpp" -d 120 -l 1,archi=st-lrwan1:sx1276+site=saclay

2. Wait for the experiment to be in the "Running" state:

In [ ]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later.

3. Open a separate terminal in the menu`File > New > Terminal` and open a terminal on your LoRa device:

4. Finally flash your new application on the LoRa device:

In [ ]:
!make IOTLAB_NODE=auto-ssh flash-only

In the terminal, you should see you device join the network with success and start sending payload to TTN.

In your TTN console, look at the messages sent by the device and in myDevices Cayenne, the dashboard starts populating with new sensors values!

### Going further

Modify the RIOT application to add the `lps22hb` atmospheric pressure sensor values to the Cayenne LPP payload:
- You can have a look at the [online documentation of the driver](http://doc.riot-os.org/group__drivers__lpsxxx.html)
- You can have a look at the [test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_lpsxxx) which provides driver API usage
- Note that the module is called `lps22hb`

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

The serial link connection through SSH will be closed automatically.